In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

sns.set(style="darkgrid")

In [ ]:
df = pd.read_csv('../data/transform/match_details_transformed.csv')
selected_columns_with_target = [
    'home_avg_goals_scored', 'away_avg_goals_scored', 'home_avg_goals_conceded', 'away_avg_goals_conceded', 'home_wins', 'away_wins', 'home_team_strength', 'away_team_strength', 'home_streak_wins', 'away_streak_wins', 'home_sum_points', 'away_sum_points', 'points_diff', 'win_eachother_home', 'win_eachother_away', 'avg_possession_home', 'avg_possession_away', 'avg_shoton_home', 'avg_shoton_away', 'result_match'
]

selected_df = df[selected_columns_with_target]

In [ ]:
# Summary statistics
summary_stats = selected_df.describe()

summary_stats

In [ ]:
# Selected columns for EDA

selected_columns = [
    'home_avg_goals_scored', 'away_avg_goals_scored', 'home_avg_goals_conceded', 'away_avg_goals_conceded', 'home_wins', 'away_wins', 'home_team_strength', 'away_team_strength', 'home_streak_wins', 'away_streak_wins', 'home_sum_points', 'away_sum_points', 'points_diff', 'win_eachother_home', 'win_eachother_away', 'avg_possession_home', 'avg_possession_away', 'avg_shoton_home', 'avg_shoton_away',
]

df_subset = selected_df[selected_columns_with_target]

# Plotting distributions for the selected columns
plt.figure(figsize=(20, 15))
for i, col in enumerate(selected_columns, 1):
    plt.subplot(5, 4, i)
    sns.histplot(df_subset[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.tight_layout()

plt.show()

In [ ]:
# - Some variables (like home_avg_goals_scored, away_avg_goals_scored, etc.) appear to be normally distributed.
# - Others, such as home_wins and away_wins, are skewed to the right.
# - Some columns (like win_eachother_home and win_eachother_away) seem to be categorical or binary, as they have distinct peaks.

In [ ]:
# Compute the correlation matrix
corr = df_subset.corr()

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.show()

In [ ]:
# Compute the correlation matrix
corr = df_subset.corr()

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.show()

In [ ]:
# Compute the correlation between the selected columns and the target variable
correlation = selected_df.corr()['result_match'].sort_values(ascending=False).drop(['result_match'])

# Plot the correlations
plt.figure(figsize=(10, 8))
sns.barplot(y=correlation.index, x=correlation.values, legend=False)
plt.title("Correlation with result_match")
plt.xlabel("Correlation Value")
plt.ylabel("Features")
plt.show()

In [ ]:
# - weigthed_wins_diff, points_diff and home_team_strength have positive correlations, implying that higher values in these features could favor the home team's win.

In [ ]:
# - away_sum_points, home_avg_goals_conceded, and away_team_strength have negative correlations, implying that higher values in these features could favor the away team's win.

In [ ]:
selected_df['result_match'] = selected_df['result_match'].map({0: 'Away win', 1: 'Draw', 2: 'Home win'})

In [ ]:
# Get top 5 positively and negatively correlated features
top_pos_correlated = correlation.index[1:6]  # Exclude the target variable itself
top_neg_correlated = correlation.index[-5:]

# Plotting the top 5 positively correlated features against result_match
plt.figure(figsize=(15, 12))
for i, feature in enumerate(top_pos_correlated, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(x="result_match", y=feature, data=selected_df, palette="Blues", hue='result_match', legend=False)
    plt.title(f"Boxplot of {feature} vs result_match")

plt.tight_layout()
plt.show()

In [ ]:
# - weigthed_wins_diff:
# For result_match = 0: The median is negative, indicating that the away team typically has a higher weighted win score than the home team for this result category (possibly an away win).
# For result_match = 1: The distribution is more centered around zero, but still slightly skewed towards positive values.
# For result_match = 2: The median value is positive, suggesting that the home team generally has a higher weighted win score compared to the away team (possibly a home win).

# - points_diff: 
# For result_match = 0: The median points_diff is negative, indicating that the away team typically has more points than the home team when the match result is 0 (possibly an away win).
# For result_match = 1: The points_diff is centered around zerosuggesting that both teams are relatively evenly matched in terms of points when the result is a draw, but still slightly skewed towards negative values.
# For result_match = 2: The median points_diff is positive, indicating that the home team generally has more points than the away team when the match result is 2 (possibly a home win).

In [ ]:
def detect_outliers(data):
    # Calculate Q1, Q2 and IQR
    Q1 = data.quantile(0.15)
    Q3 = data.quantile(0.85)
    IQR = Q3 - Q1
    # Define bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Identify outliers
    return ((data < lower_bound) | (data > upper_bound)).sum()

outliers_count = detect_outliers(selected_df.drop(columns="result_match"))
outliers_count

In [ ]:
# Plot interactions between goal-related features and team strength
plt.figure(figsize=(15, 10))

# Interaction between home_avg_goals_scored and home_team_strength
plt.subplot(2, 2, 1)
sns.scatterplot(x='home_avg_goals_scored', y='home_team_strength', hue='result_match', data=selected_df, palette='deep', style='result_match')
plt.title('Home Goals Scored vs Home Team Strength')

# Interaction between away_avg_goals_scored and away_team_strength
plt.subplot(2, 2, 2)
sns.scatterplot(x='away_avg_goals_scored', y='away_team_strength', hue='result_match', data=selected_df, palette='deep', style='result_match')
plt.title('Away Goals Scored vs Away Team Strength')

# Interaction between home_avg_goals_scored and away_avg_goals_scored
plt.subplot(2, 2, 3)
sns.scatterplot(x='home_avg_goals_scored', y='away_avg_goals_scored', hue='result_match', data=selected_df, palette='deep', style='result_match')
plt.title('Home Goals Scored vs Away Goals Scored')

# Interaction between home_team_strength and away_team_strength
plt.subplot(2, 2, 4)
sns.scatterplot(x='home_team_strength', y='away_team_strength', hue='result_match', data=selected_df, palette='deep', style='result_match')
plt.title('Home Team Strength vs Away Team Strength')

plt.tight_layout()
plt.show()

In [ ]:
# - Home Goals Scored vs Home Team Strength:

# There's a general trend suggesting that stronger teams (with higher strength values) tend to score more goals on average.
# The color hue (representing match results) indicates that when the home team scores more goals, they are more likely to win.

# - Away Goals Scored vs Away Team Strength:

# Similar to home teams, there's a trend where stronger away teams tend to score more goals on average.
# Matches where the away team scores more goals are more likely to result in an away win.

# - Home Goals Scored vs Away Goals Scored:

# Trend suggests that when home_avg_goals_scored is bigger than away_avg_goals_scored than home team win.
# Points above the diagonal line indicate home wins, and points below the diagonal line suggest away wins.

# - Home Team Strength vs Away Team Strength:

# There's a concentration of points around the center, indicating that many matches are between teams of similar strengths.
# The color hue shows that when the away team is stronger, they are more likely to win, and vice versa.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Split the data into features and target
X = selected_df.drop(columns="result_match")
y = selected_df["result_match"]

# Fit the model
rf.fit(X, y)

# Get feature importances
feature_importances = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)

# Plot feature importances
plt.figure(figsize=(12, 8))
feature_importances.plot(kind='barh', color='skyblue')
plt.title('Feature Importances using Random Forest')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Sort by VIF values in descending order
vif_sorted = vif_data.sort_values(by="VIF", ascending=False)
vif_sorted

In [ ]:
# Features such as avg_shoton_away, avg_possession_home, avg_shoton_home, home_sum_points, away_sum_points, points_diff, and avg_possession_away have infinite VIF values, suggesting they are perfectly (or near-perfectly) correlated with other features in the dataset. These features might need to be dropped or combined with others to avoid multicollinearity issues.
# Features away_wins, home_wins, away_team_strength, and home_team_strength have high VIF values, indicating potential multicollinearity with other features.

In [ ]:
# 1. Match Outcome by Team Strength

# Creating a new column to indicate which team has higher strength
selected_df['stronger_team'] = 'draw'
selected_df.loc[selected_df['home_team_strength'] > selected_df['away_team_strength'], 'stronger_team'] = 'home'
selected_df.loc[selected_df['home_team_strength'] < selected_df['away_team_strength'], 'stronger_team'] = 'away'

# Mapping result_match to outcomes
outcomes = {0: 'away_win', 1: 'draw', 2: 'home_win'}
selected_df['match_outcome'] = selected_df['result_match'].map(outcomes)

# Counting outcomes based on which team was stronger
strength_outcome = selected_df.groupby(['stronger_team', 'match_outcome']).size().unstack()

In [ ]:
# Teams deemed "stronger" (either the home or away team) tend to win more often,
# as evidenced by the larger segments for "home_win" and "away_win" respectively.
strength_outcome

In [ ]:
# 2. Effect of Streaks

# Filtering matches where home streak is bigger than away streak 
streak_threshold = 2
streak_data = selected_df[selected_df['home_streak_wins'] > selected_df['away_streak_wins']]

# Counting outcomes based on streaks
streak_outcomes = streak_data.groupby(['match_outcome']).size()

streak_outcomes.plot(kind='pie', figsize=(8, 6), colormap='Set3', autopct='%1.1f%%', startangle=140, wedgeprops=dict(width=0.3))
plt.title('Match Outcomes when a home team is on a Streak')
plt.ylabel('')  # Removing the default ylabel
plt.tight_layout()
plt.show()

In [ ]:
# Plot distribution for 'result_match' (Categorical variable)
plt.figure(figsize=(8, 4))
sns.countplot(x='result_match', data=selected_df)
plt.title('Distribution of Match Results')
plt.show()